<a href="https://colab.research.google.com/github/Summarizer-cb/Gas_Energy_Summarization/blob/main/%5B%EC%B0%BD%EC%9A%B0%5D1125_TOKENIZING(%EC%98%A4%EB%A5%98%EC%88%98%EC%A0%95)%2BROUGE(aihub%2Cnaver).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

### 문장구분

In [ ]:
df_aihub = pd.read_csv("/<your_dir>/AIHUB_GSG_dataset_371263.csv")
df_naver = pd.read_csv("/<your_dir>/네이버뉴스_GSG_dataset_200061.csv")

In [ ]:
# 문장구분 <GSG> -> <SEP>
# 빈공간 제외 (2개연속 or 맨처음 or 맨마지막)
df_naver['content'].progress_map(lambda x : x.replace("\ue05b",""))
df_naver['title'].progress_map(lambda x : x.replace("\ue05b",""))
df_aihub['content'] = df_aihub['content'].progress_map(lambda x : '<SEP>'.join([s.strip() for s in x.split('<GSG>') if s!='']))
df_naver['content'] = df_naver['content'].progress_map(lambda x : '<SEP>'.join([s.strip() for s in x.split('<GSG>') if s!='']))

  0%|          | 0/200061 [00:00<?, ?it/s]

  0%|          | 0/200061 [00:00<?, ?it/s]

  0%|          | 0/371263 [00:00<?, ?it/s]

  0%|          | 0/200061 [00:00<?, ?it/s]

### 토크나이징

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart")
added_token_num = tokenizer.add_special_tokens({"additional_special_tokens":["<GSG>"]})
tokenizer.special_tokens_map

{'additional_special_tokens': ['<GSG>'],
 'bos_token': '<s>',
 'cls_token': '<s>',
 'eos_token': '</s>',
 'mask_token': '<mask>',
 'pad_token': '<pad>',
 'sep_token': '</s>',
 'unk_token': '<unk>'}

#### aihub

In [ ]:
df_aihub['title_tok'] = df_aihub['title'].progress_map(lambda x : tokenizer(x)['input_ids'])
df_aihub['content_tok'] = df_aihub['content'].progress_map(lambda x : tokenizer(x.split('<SEP>'))['input_ids'])
df_aihub['abstractive_tok'] = df_aihub['abstractive'].progress_map(lambda x : tokenizer(x)['input_ids'])
df_aihub['content_tok_len'] = df_aihub['content_tok'].progress_map(lambda x : sum([len(n) for n in x]))

  0%|          | 0/371263 [00:00<?, ?it/s]

  0%|          | 0/371263 [00:00<?, ?it/s]

  0%|          | 0/371263 [00:00<?, ?it/s]

  0%|          | 0/371263 [00:00<?, ?it/s]

#### naver

In [ ]:
df_naver['title_tok'] = df_naver['title'].progress_map(lambda x : tokenizer(x)['input_ids'])
df_naver['content_tok'] = df_naver['content'].progress_map(lambda x : tokenizer(x.split('<SEP>'))['input_ids'])
df_naver['content_tok_len'] = df_naver['content_tok'].progress_map(lambda x : sum([len(n) for n in x]))

  0%|          | 0/200061 [00:00<?, ?it/s]

  0%|          | 0/200061 [00:00<?, ?it/s]

  0%|          | 0/200061 [00:00<?, ?it/s]

In [ ]:
df_naver = df_naver.drop(columns=['url'])

#### 확인


In [ ]:
def test(x):
  for n in x:
    if n==[]:
      print(x)
df_aihub['content_tok'].apply(test)
df_naver['content_tok'].apply(test)
print()

[[17332, 16819, 18378, 10314, 14370, 14239, 14703, 20389, 14049, 15736, 8981, 16090, 16889, 14652, 15111, 14646, 14368, 14466, 15474, 15131, 14280, 15282, 15131, 12024, 16757, 9229, 20459, 17237, 14518, 15131, 14931, 239, 17332, 15131, 14931, 15092, 17308, 18536, 19553], [9967, 9715, 12005, 14164, 12034, 23609, 14076, 14665, 19505, 19663, 15029, 14130], [15188, 18538, 15962, 14043, 1612, 15104, 12007, 17949, 23919, 17796, 15631, 14356, 20194, 14157, 27796, 16433, 22164, 15046, 13230, 8981, 15098, 14528, 15358, 12613, 11841, 14280, 25861, 15283, 15311, 15070, 9904, 14553, 14236, 15394, 16247], [15131, 14790, 14518, 18378, 17020, 17521, 9866, 14264, 14261, 14053, 15621, 14380, 16247], [15666, 14319, 16420, 14578, 16739, 14134, 26656, 14559, 14457, 22164, 11786, 14105, 8985, 13848, 12007, 16262, 9698, 14598, 9866, 14742, 16730, 15964], [11721, 11268, 22294, 22139, 14812, 20111, 19666, 20182, 14191, 27661], [9229, 10290, 14279, 16759, 21939, 22164, 15399, 12005, 21447, 14997, 15544, 22564]

#### 확인


In [ ]:
df_aihub

,title,content,abstractive,character_len,sentence_len,title_tok,content_tok,abstractive_tok,content_tok_len
0,"문대통령 ""5G는 4차산업혁명 시대의 고속도로""","문 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 ""5G 시대...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,974,12,"[10671, 19774, 14050, 252, 270, 9698, 21963, 1...","[[10671, 15606, 16635, 14245, 16653, 9115, 153...","[255, 12041, 22876, 14739, 14144, 270, 20769, ...",364
1,"""손학규 물러나라""...바른계, 최고위 회의 '보이콧'",당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.<SEP>바른미래당이 8일 ...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,851,11,"[233, 11323, 28103, 17680, 14786, 233, 14476, ...","[[9770, 28266, 15484, 16534, 16005, 20152, 203...","[255, 12041, 20152, 20324, 9770, 16507, 11936,...",323
2,"""마치 한 장소처럼"" ... 5G로 서울·부산·광주 원격협연",지난 3일 한국이 세계 최초로 5세대(5G) 이동통신 서비스를 상용화한 것을 기념하...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,1050,16,"[233, 10496, 12863, 14036, 22318, 14400, 233, ...","[[25560, 16356, 21724, 14370, 17293, 14144, 16...","[25560, 16356, 21724, 14370, 14522, 14144, 162...",432
3,"""이웃도 모르는데, 이게 서울 미래냐""...'고층 재개발 요구' 작심 비판한 박원순",박원순 서울시장이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리를 쏟아냈다...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,702,9,"[233, 12034, 11924, 9866, 16200, 15133, 17032,...","[[10770, 11936, 11374, 16422, 14416, 16635, 14...","[10770, 11936, 11374, 16422, 14266, 16635, 164...",279
4,"""SK·알파벳 벤치마킹해야""...KB운용, KMH에 주주서한",기업 지배구조 개선 등을 통해 높은 수익률을 올리는 것을 목표로 하는 공모펀드 'K...,주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,1275,16,"[233, 24514, 373, 11703, 13369, 10860, 27742, ...","[[14451, 16614, 16305, 15098, 14382, 14232, 14...","[12258, 16387, 12863, 14219, 12922, 15116, 181...",525
...,...,...,...,...,...,...,...,...,...
371258,잇단 항공 안전사고와 보잉 공포... 안일한 대응은 금물,항공기 안전사고가 잇따르면서 국민들의 불안감이 커지고 있다.<SEP>지난 25 일 ...,잇단 항공기 안전사고로 불안감이 커지는 만큼 부주위가 대형참사로 이어질 수 있다는 ...,1192,24,"[12051, 9758, 16788, 15062, 16447, 11863, 1404...","[[16732, 9264, 15062, 16447, 8981, 17684, 2992...","[12051, 9758, 27690, 15062, 16447, 10338, 1614...",467
371259,트럼프의 ‘동맹 비즈니스’,도널드 트럼프 미국 대통령의 동맹 인식에 심각한 의문을 제기해야 할 상황이 벌어지고...,지난주 방위비 협상에서 50억 달러(6조원) 수준으로 과도하게 인상된 분담금을 요구...,1191,25,"[13328, 18067, 12024, 14143, 25196, 20159, 1543]","[[9866, 28260, 18751, 14287, 15832, 22450, 160...","[25560, 12258, 25462, 11028, 14405, 17357, 154...",464
371260,"이해찬 대표, 조국 사태 송구하다고 말했지만...","더불어민주당 이해찬 대표가 30 일 조국 사태와 관련해 ""국민 여러분께 매우 송구하...","더불어민주당 이해찬 대표가 30일 조국 사태를 검찰 개혁과 연관지어 설명하면서, 국...",1287,21,"[12034, 13607, 12615, 26148, 29641, 16211, 147...","[[9806, 16406, 18979, 15001, 12615, 18173, 146...","[9806, 16406, 18979, 15001, 12615, 18173, 1696...",468
371261,‘비정규직 제로’ 정부 약속 무색하게 만든 비정규직 폭증 통계,강신욱 통계청장이 29 일 서울 종로구 정부서울청사 브리핑룸에서 2019 년 8 월...,올해 경제활동인구조사의 근로형태별 부가조사에서 집계된 비정규직 노동자 수는 전년대비...,1207,14,"[1542, 11028, 12141, 16683, 29262, 1543, 14295...","[[9000, 11467, 11912, 19672, 12681, 14416, 156...","[19973, 14407, 16095, 12037, 16305, 14643, 162...",459


In [ ]:
df_naver

,title,content,character_len,sentence_len,title_tok,content_tok,content_tok_len
0,경기 부진·코로나19에 2년 연속 저물가,올해 소비자물가상승률 0.5% 사상 처음 2년 연속 0%대 상승 올해 소비자물가가 ...,1330,23,"[15398, 17750, 373, 12972, 10338, 9495, 15874,...","[[19973, 17612, 28292, 21480, 10432, 14894, 18...",523
1,"""국내 인기 웹사이트 친환경 점수는 낙제점""..그린피스","국내 주요 데이터센터들이 재생에너지 사용에 대한 의지 부족<SEP>그린피스, 재생가...",1507,17,"[233, 23436, 16808, 27499, 18043, 14244, 15208...","[[23436, 14979, 17360, 14671, 14108, 24809, 19...",566
2,"귀뚜라미, '가스 열병합발전' 나선다",한국도시가스협회와 사업제휴...시장확대 본격화 귀뚜라미(대표 송경석)가 가스 열병합...,848,14,"[9212, 10138, 10213, 23547, 14063, 18982, 1418...","[[15274, 15429, 18982, 16212, 11863, 14379, 12...",353
3,"경동도시가스, 선암호수공원서 꽃심기 봉사",경동도시가스 사랑나눔 봉사단은 31일 선암호수공원에서 봄맞이 꽃 심기 봉사활동을 실...,356,6,"[9085, 9879, 15429, 8981, 16603, 14084, 11711,...","[[9085, 9879, 15429, 18982, 14733, 25594, 1778...",167
4,"""강릉 과학단지 수소폭발 재발방지"" 안전방안·법 제정 추진",정부가 지난 5월 발생한 강릉 과학단지 수소탱크 폭발사고와 같은 일이 또 벌어지는 ...,1223,12,"[233, 9000, 10452, 15840, 17481, 14032, 11319,...","[[20490, 14141, 15503, 17419, 22195, 15840, 17...",512
...,...,...,...,...,...,...,...
200056,<새책>'에너지 정치경제학' 출판 다이제스트,에너지 정치경제학/이재호 지음/석전력수급도 역시 비상이다.<SEP>원자력발전소 때문...,1412,20,"[259, 11229, 12632, 261, 15565, 15488, 14499, ...","[[19329, 14499, 14714, 13587, 246, 12034, 1208...",625
200057,12월 소비자물가 4.2%.. 연간 4.0% 상승,이달 소비자물가가 1년전보다 4.2% 상승했습니다.<SEP>통계청이 발표한 소비자 ...,450,6,"[17206, 11940, 17612, 28292, 14136, 20742, 141...","[[12034, 9760, 17612, 10675, 15305, 15623, 121...",189
200058,"김주호 울산시 교육혁신도시협력관 ""신재생·소재 등 R&D센터,울산 우정혁신도시에 유치""","""신재생에너지 및 소재분야 연구개발(R&D)기관을 우정혁신도시 산.학.연 클러스터에...",1239,10,"[9275, 12258, 13699, 16176, 11465, 14468, 1938...","[[233, 11467, 21982, 19329, 14280, 16516, 1830...",510
200059,가스공-가스안전공-경동나비엔 등 취약계층 주거복지 통합 지원 협약,"가스공 올해 22억 투입, 복지시설 난방 개선 등한국가스공사는 10월 30일 경기도...",1009,10,"[18982, 9115, 244, 18982, 16239, 9115, 244, 90...","[[18982, 9115, 14516, 15271, 11764, 16490, 243...",405


In [ ]:
df_aihub.to_csv("/<your_dir>/token_AIHUB_371263.csv",index=False)
df_naver.to_csv("/<your_dir>/token_NAVER_200061.csv",index=False)

## ROUGE

In [1]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

### AIHUB

In [2]:
df_aihub = pd.read_csv("/<your_dir>/token_AIHUB_371263.csv")
df_aihub['title_tok'] = df_aihub['title_tok'].progress_map(eval)
df_aihub['content_tok'] = df_aihub['content_tok'].progress_map(eval)
df_aihub['abstractive_tok'] = df_aihub['abstractive_tok'].progress_map(eval)
df_aihub

  0%|          | 0/371263 [00:00<?, ?it/s]

  0%|          | 0/371263 [00:00<?, ?it/s]

  0%|          | 0/371263 [00:00<?, ?it/s]

,title,content,abstractive,character_len,sentence_len,title_tok,content_tok,abstractive_tok,content_tok_len
0,"문대통령 ""5G는 4차산업혁명 시대의 고속도로""","문 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 ""5G 시대...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,974,12,"[10671, 19774, 14050, 252, 270, 9698, 21963, 1...","[[10671, 15606, 16635, 14245, 16653, 9115, 153...","[255, 12041, 22876, 14739, 14144, 270, 20769, ...",364
1,"""손학규 물러나라""...바른계, 최고위 회의 '보이콧'",당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.<SEP>바른미래당이 8일 ...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,851,11,"[233, 11323, 28103, 17680, 14786, 233, 14476, ...","[[9770, 28266, 15484, 16534, 16005, 20152, 203...","[255, 12041, 20152, 20324, 9770, 16507, 11936,...",323
2,"""마치 한 장소처럼"" ... 5G로 서울·부산·광주 원격협연",지난 3일 한국이 세계 최초로 5세대(5G) 이동통신 서비스를 상용화한 것을 기념하...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,1050,16,"[233, 10496, 12863, 14036, 22318, 14400, 233, ...","[[25560, 16356, 21724, 14370, 17293, 14144, 16...","[25560, 16356, 21724, 14370, 14522, 14144, 162...",432
3,"""이웃도 모르는데, 이게 서울 미래냐""...'고층 재개발 요구' 작심 비판한 박원순",박원순 서울시장이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리를 쏟아냈다...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,702,9,"[233, 12034, 11924, 9866, 16200, 15133, 17032,...","[[10770, 11936, 11374, 16422, 14416, 16635, 14...","[10770, 11936, 11374, 16422, 14266, 16635, 164...",279
4,"""SK·알파벳 벤치마킹해야""...KB운용, KMH에 주주서한",기업 지배구조 개선 등을 통해 높은 수익률을 올리는 것을 목표로 하는 공모펀드 'K...,주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,1275,16,"[233, 24514, 373, 11703, 13369, 10860, 27742, ...","[[14451, 16614, 16305, 15098, 14382, 14232, 14...","[12258, 16387, 12863, 14219, 12922, 15116, 181...",525
...,...,...,...,...,...,...,...,...,...
371258,잇단 항공 안전사고와 보잉 공포... 안일한 대응은 금물,항공기 안전사고가 잇따르면서 국민들의 불안감이 커지고 있다.<SEP>지난 25 일 ...,잇단 항공기 안전사고로 불안감이 커지는 만큼 부주위가 대형참사로 이어질 수 있다는 ...,1192,24,"[12051, 9758, 16788, 15062, 16447, 11863, 1404...","[[16732, 9264, 15062, 16447, 8981, 17684, 2992...","[12051, 9758, 27690, 15062, 16447, 10338, 1614...",467
371259,트럼프의 ‘동맹 비즈니스’,도널드 트럼프 미국 대통령의 동맹 인식에 심각한 의문을 제기해야 할 상황이 벌어지고...,지난주 방위비 협상에서 50억 달러(6조원) 수준으로 과도하게 인상된 분담금을 요구...,1191,25,"[13328, 18067, 12024, 14143, 25196, 20159, 1543]","[[9866, 28260, 18751, 14287, 15832, 22450, 160...","[25560, 12258, 25462, 11028, 14405, 17357, 154...",464
371260,"이해찬 대표, 조국 사태 송구하다고 말했지만...","더불어민주당 이해찬 대표가 30 일 조국 사태와 관련해 ""국민 여러분께 매우 송구하...","더불어민주당 이해찬 대표가 30일 조국 사태를 검찰 개혁과 연관지어 설명하면서, 국...",1287,21,"[12034, 13607, 12615, 26148, 29641, 16211, 147...","[[9806, 16406, 18979, 15001, 12615, 18173, 146...","[9806, 16406, 18979, 15001, 12615, 18173, 1696...",468
371261,‘비정규직 제로’ 정부 약속 무색하게 만든 비정규직 폭증 통계,강신욱 통계청장이 29 일 서울 종로구 정부서울청사 브리핑룸에서 2019 년 8 월...,올해 경제활동인구조사의 근로형태별 부가조사에서 집계된 비정규직 노동자 수는 전년대비...,1207,14,"[1542, 11028, 12141, 16683, 29262, 1543, 14295...","[[9000, 11467, 11912, 19672, 12681, 14416, 156...","[19973, 14407, 16095, 12037, 16305, 14643, 162...",459


### NAVER

In [3]:
df_naver = pd.read_csv("/<your_dir>/token_NAVER_200061.csv")
df_naver['title_tok'] = df_naver['title_tok'].progress_map(eval)
df_naver['content_tok'] = df_naver['content_tok'].progress_map(eval)
df_naver

  0%|          | 0/200061 [00:00<?, ?it/s]

  0%|          | 0/200061 [00:00<?, ?it/s]

,title,content,character_len,sentence_len,title_tok,content_tok,content_tok_len
0,경기 부진·코로나19에 2년 연속 저물가,올해 소비자물가상승률 0.5% 사상 처음 2년 연속 0%대 상승 올해 소비자물가가 ...,1330,23,"[15398, 17750, 373, 12972, 10338, 9495, 15874,...","[[19973, 17612, 28292, 21480, 10432, 14894, 18...",523
1,"""국내 인기 웹사이트 친환경 점수는 낙제점""..그린피스","국내 주요 데이터센터들이 재생에너지 사용에 대한 의지 부족<SEP>그린피스, 재생가...",1507,17,"[233, 23436, 16808, 27499, 18043, 14244, 15208...","[[23436, 14979, 17360, 14671, 14108, 24809, 19...",566
2,"귀뚜라미, '가스 열병합발전' 나선다",한국도시가스협회와 사업제휴...시장확대 본격화 귀뚜라미(대표 송경석)가 가스 열병합...,848,14,"[9212, 10138, 10213, 23547, 14063, 18982, 1418...","[[15274, 15429, 18982, 16212, 11863, 14379, 12...",353
3,"경동도시가스, 선암호수공원서 꽃심기 봉사",경동도시가스 사랑나눔 봉사단은 31일 선암호수공원에서 봄맞이 꽃 심기 봉사활동을 실...,356,6,"[9085, 9879, 15429, 8981, 16603, 14084, 11711,...","[[9085, 9879, 15429, 18982, 14733, 25594, 1778...",167
4,"""강릉 과학단지 수소폭발 재발방지"" 안전방안·법 제정 추진",정부가 지난 5월 발생한 강릉 과학단지 수소탱크 폭발사고와 같은 일이 또 벌어지는 ...,1223,12,"[233, 9000, 10452, 15840, 17481, 14032, 11319,...","[[20490, 14141, 15503, 17419, 22195, 15840, 17...",512
...,...,...,...,...,...,...,...
200056,<새책>'에너지 정치경제학' 출판 다이제스트,에너지 정치경제학/이재호 지음/석전력수급도 역시 비상이다.<SEP>원자력발전소 때문...,1412,20,"[259, 11229, 12632, 261, 15565, 15488, 14499, ...","[[19329, 14499, 14714, 13587, 246, 12034, 1208...",625
200057,12월 소비자물가 4.2%.. 연간 4.0% 상승,이달 소비자물가가 1년전보다 4.2% 상승했습니다.<SEP>통계청이 발표한 소비자 ...,450,6,"[17206, 11940, 17612, 28292, 14136, 20742, 141...","[[12034, 9760, 17612, 10675, 15305, 15623, 121...",189
200058,"김주호 울산시 교육혁신도시협력관 ""신재생·소재 등 R&D센터,울산 우정혁신도시에 유치""","""신재생에너지 및 소재분야 연구개발(R&D)기관을 우정혁신도시 산.학.연 클러스터에...",1239,10,"[9275, 12258, 13699, 16176, 11465, 14468, 1938...","[[233, 11467, 21982, 19329, 14280, 16516, 1830...",510
200059,가스공-가스안전공-경동나비엔 등 취약계층 주거복지 통합 지원 협약,"가스공 올해 22억 투입, 복지시설 난방 개선 등한국가스공사는 10월 30일 경기도...",1009,10,"[18982, 9115, 244, 18982, 16239, 9115, 244, 90...","[[18982, 9115, 14516, 15271, 11764, 16490, 243...",405


### ROUGE FUNC

In [4]:
from itertools import chain
from statistics import mean

def rouge_score(docs, title, abstractive=None):
  # rouge 1 
  results = []
  N = len(docs)
  for i in range(N):
    doc = docs.pop(i) # 하나뽑음
    if doc==[]:
      results.append((i,0))
      docs.insert(i,doc)
      continue

    if abstractive:
      rem = title # remain 나머지
      rem.extend(abstractive) # remain + title
    else :
      rem = list(chain.from_iterable(docs)) # remain 나머지
      rem.append(title) # remain + title

    pre_match = [1 if x in rem else 0 for x in doc]
    rec_match = [1 if x in doc else 0 for x in rem]

    assert pre_match!=[],f"{pre_match}, {doc}, {rem}"
    assert rec_match!=[],f"{rec_match}, {doc}, {rem}"

    pre = mean(pre_match)  # precision
    rec = mean(rec_match)  # recall

    if (pre+rec) == 0:
      f1score = 0
    else :
      f1score = 2*((pre*rec)/(pre+rec)) #f1구함
    results.append((i,f1score)) #결과더해줌

    docs.insert(i,doc)
  assert len(results)==N
  return results

In [5]:
sample_aihub = df_aihub.iloc[0]
rouge_score(sample_aihub['content_tok'],sample_aihub['title_tok'],sample_aihub['abstractive_tok'])

[(0, 0.437807679684936),
 (1, 0.2849389416553596),
 (2, 0.4622222222222222),
 (3, 0.1731399157697707),
 (4, 0.09817893903404593),
 (5, 0.047244094488188976),
 (6, 0.15648510382184772),
 (7, 0.16496308915388982),
 (8, 0.04173531026908292),
 (9, 0.16565656565656567),
 (10, 0.0639561443581544),
 (11, 0.0453224869262057)]

In [6]:
def get_top_k_sentence(docs, title, abstractive=None,
                       verbose=False):
  results = rouge_score(docs, title, abstractive=None)
  results = sorted(results,key=lambda tup: tup[1])

  top_k_len = int(len(results)*0.3)
  if top_k_len<1 : 
    top_k_len = 1
  rouge_index = results[:top_k_len]
  if verbose:
    print("입력된 문장의 갯수 :",len(docs))
    print("선정된 문장의 갯수 :",len(rouge_index))
    print("선정된 문장및 루지 : ",rouge_index)
  rouge_index = [ri[0] for ri in rouge_index]
  return rouge_index


In [7]:
sample_aihub = df_aihub.iloc[0]
get_top_k_sentence(sample_aihub['content_tok'],sample_aihub['title_tok'],sample_aihub['abstractive_tok'],True)

입력된 문장의 갯수 : 12
선정된 문장의 갯수 : 3
선정된 문장및 루지 :  [(8, 0.06597549481621112), (10, 0.07017543859649122), (5, 0.07594936708860758)]


[8, 10, 5]

In [8]:
sample_naver = df_naver.iloc[0]
get_top_k_sentence(sample_naver['content_tok'],sample_naver['title_tok'],verbose=True)

입력된 문장의 갯수 : 22
선정된 문장의 갯수 : 6
선정된 문장및 루지 :  [(14, 0.05585292064230859), (4, 0.059186189889025895), (6, 0.06227106227106227), (7, 0.08175905853205327), (11, 0.08208955223880597), (9, 0.08410904255319149)]


[14, 4, 6, 7, 11, 9]

## ADD ROUGE COLUMN


In [9]:
temp_aihub = []
for i in tqdm(range(len(df_aihub))):
  sample_aihub = df_aihub.iloc[i]
  rouge_index = get_top_k_sentence(sample_aihub['content_tok'],sample_aihub['title_tok'],sample_aihub['abstractive_tok'])
  temp_aihub.append(rouge_index)
  assert (i+1)==len(temp_aihub)

  0%|          | 0/371263 [00:00<?, ?it/s]

In [12]:
df_aihub['Gap_sentence'] = temp_aihub
df_aihub

,title,content,abstractive,character_len,sentence_len,title_tok,content_tok,abstractive_tok,content_tok_len,Gap_sentence
0,"문대통령 ""5G는 4차산업혁명 시대의 고속도로""","문 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 ""5G 시대...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,974,12,"[10671, 19774, 14050, 252, 270, 9698, 21963, 1...","[[10671, 15606, 16635, 14245, 16653, 9115, 153...","[255, 12041, 22876, 14739, 14144, 270, 20769, ...",364,"[8, 10, 5]"
1,"""손학규 물러나라""...바른계, 최고위 회의 '보이콧'",당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.<SEP>바른미래당이 8일 ...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,851,11,"[233, 11323, 28103, 17680, 14786, 233, 14476, ...","[[9770, 28266, 15484, 16534, 16005, 20152, 203...","[255, 12041, 20152, 20324, 9770, 16507, 11936,...",323,"[1, 3, 4]"
2,"""마치 한 장소처럼"" ... 5G로 서울·부산·광주 원격협연",지난 3일 한국이 세계 최초로 5세대(5G) 이동통신 서비스를 상용화한 것을 기념하...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,1050,16,"[233, 10496, 12863, 14036, 22318, 14400, 233, ...","[[25560, 16356, 21724, 14370, 17293, 14144, 16...","[25560, 16356, 21724, 14370, 14522, 14144, 162...",432,"[3, 2, 15, 12]"
3,"""이웃도 모르는데, 이게 서울 미래냐""...'고층 재개발 요구' 작심 비판한 박원순",박원순 서울시장이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리를 쏟아냈다...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,702,9,"[233, 12034, 11924, 9866, 16200, 15133, 17032,...","[[10770, 11936, 11374, 16422, 14416, 16635, 14...","[10770, 11936, 11374, 16422, 14266, 16635, 164...",279,"[6, 3]"
4,"""SK·알파벳 벤치마킹해야""...KB운용, KMH에 주주서한",기업 지배구조 개선 등을 통해 높은 수익률을 올리는 것을 목표로 하는 공모펀드 'K...,주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,1275,16,"[233, 24514, 373, 11703, 13369, 10860, 27742, ...","[[14451, 16614, 16305, 15098, 14382, 14232, 14...","[12258, 16387, 12863, 14219, 12922, 15116, 181...",525,"[5, 14, 12, 8]"
...,...,...,...,...,...,...,...,...,...,...
371258,잇단 항공 안전사고와 보잉 공포... 안일한 대응은 금물,항공기 안전사고가 잇따르면서 국민들의 불안감이 커지고 있다.<SEP>지난 25 일 ...,잇단 항공기 안전사고로 불안감이 커지는 만큼 부주위가 대형참사로 이어질 수 있다는 ...,1192,24,"[12051, 9758, 16788, 15062, 16447, 11863, 1404...","[[16732, 9264, 15062, 16447, 8981, 17684, 2992...","[12051, 9758, 27690, 15062, 16447, 10338, 1614...",467,"[22, 2, 11, 0, 17, 13, 23]"
371259,트럼프의 ‘동맹 비즈니스’,도널드 트럼프 미국 대통령의 동맹 인식에 심각한 의문을 제기해야 할 상황이 벌어지고...,지난주 방위비 협상에서 50억 달러(6조원) 수준으로 과도하게 인상된 분담금을 요구...,1191,25,"[13328, 18067, 12024, 14143, 25196, 20159, 1543]","[[9866, 28260, 18751, 14287, 15832, 22450, 160...","[25560, 12258, 25462, 11028, 14405, 17357, 154...",464,"[10, 8, 17, 2, 3, 11, 22]"
371260,"이해찬 대표, 조국 사태 송구하다고 말했지만...","더불어민주당 이해찬 대표가 30 일 조국 사태와 관련해 ""국민 여러분께 매우 송구하...","더불어민주당 이해찬 대표가 30일 조국 사태를 검찰 개혁과 연관지어 설명하면서, 국...",1287,21,"[12034, 13607, 12615, 26148, 29641, 16211, 147...","[[9806, 16406, 18979, 15001, 12615, 18173, 146...","[9806, 16406, 18979, 15001, 12615, 18173, 1696...",468,"[9, 8, 15, 19, 10, 0]"
371261,‘비정규직 제로’ 정부 약속 무색하게 만든 비정규직 폭증 통계,강신욱 통계청장이 29 일 서울 종로구 정부서울청사 브리핑룸에서 2019 년 8 월...,올해 경제활동인구조사의 근로형태별 부가조사에서 집계된 비정규직 노동자 수는 전년대비...,1207,14,"[1542, 11028, 12141, 16683, 29262, 1543, 14295...","[[9000, 11467, 11912, 19672, 12681, 14416, 156...","[19973, 14407, 16095, 12037, 16305, 14643, 162...",459,"[10, 9, 11, 3]"


In [13]:
df_aihub.to_csv("/<your_dir>/GSG_AIHUB_371263.csv",index=False)

In [14]:
temp_naver = []
for i in tqdm(range(len(df_naver))):
  sample_naver = df_naver.iloc[i]
  rouge_index = get_top_k_sentence(sample_naver['content_tok'],sample_naver['title_tok'])
  temp_naver.append(rouge_index)
  assert (i+1)==len(temp_naver)

  0%|          | 0/200061 [00:00<?, ?it/s]

In [17]:
df_naver['Gap_sentence'] = temp_naver
df_naver

,title,content,character_len,sentence_len,title_tok,content_tok,content_tok_len,Gap_sentence
0,경기 부진·코로나19에 2년 연속 저물가,올해 소비자물가상승률 0.5% 사상 처음 2년 연속 0%대 상승 올해 소비자물가가 ...,1330,23,"[15398, 17750, 373, 12972, 10338, 9495, 15874,...","[[19973, 17612, 28292, 21480, 10432, 14894, 18...",523,"[14, 4, 6, 7, 11, 9]"
1,"""국내 인기 웹사이트 친환경 점수는 낙제점""..그린피스","국내 주요 데이터센터들이 재생에너지 사용에 대한 의지 부족<SEP>그린피스, 재생가...",1507,17,"[233, 23436, 16808, 27499, 18043, 14244, 15208...","[[23436, 14979, 17360, 14671, 14108, 24809, 19...",566,"[9, 15, 11, 2]"
2,"귀뚜라미, '가스 열병합발전' 나선다",한국도시가스협회와 사업제휴...시장확대 본격화 귀뚜라미(대표 송경석)가 가스 열병합...,848,14,"[9212, 10138, 10213, 23547, 14063, 18982, 1418...","[[15274, 15429, 18982, 16212, 11863, 14379, 12...",353,"[13, 5, 10, 8]"
3,"경동도시가스, 선암호수공원서 꽃심기 봉사",경동도시가스 사랑나눔 봉사단은 31일 선암호수공원에서 봄맞이 꽃 심기 봉사활동을 실...,356,6,"[9085, 9879, 15429, 8981, 16603, 14084, 11711,...","[[9085, 9879, 15429, 18982, 14733, 25594, 1778...",167,[3]
4,"""강릉 과학단지 수소폭발 재발방지"" 안전방안·법 제정 추진",정부가 지난 5월 발생한 강릉 과학단지 수소탱크 폭발사고와 같은 일이 또 벌어지는 ...,1223,12,"[233, 9000, 10452, 15840, 17481, 14032, 11319,...","[[20490, 14141, 15503, 17419, 22195, 15840, 17...",512,"[4, 2, 8]"
...,...,...,...,...,...,...,...,...
200056,<새책>'에너지 정치경제학' 출판 다이제스트,에너지 정치경제학/이재호 지음/석전력수급도 역시 비상이다.<SEP>원자력발전소 때문...,1412,20,"[259, 11229, 12632, 261, 15565, 15488, 14499, ...","[[19329, 14499, 14714, 13587, 246, 12034, 1208...",625,"[17, 1, 11, 12, 3, 8]"
200057,12월 소비자물가 4.2%.. 연간 4.0% 상승,이달 소비자물가가 1년전보다 4.2% 상승했습니다.<SEP>통계청이 발표한 소비자 ...,450,6,"[17206, 11940, 17612, 28292, 14136, 20742, 141...","[[12034, 9760, 17612, 10675, 15305, 15623, 121...",189,[5]
200058,"김주호 울산시 교육혁신도시협력관 ""신재생·소재 등 R&D센터,울산 우정혁신도시에 유치""","""신재생에너지 및 소재분야 연구개발(R&D)기관을 우정혁신도시 산.학.연 클러스터에...",1239,10,"[9275, 12258, 13699, 16176, 11465, 14468, 1938...","[[233, 11467, 21982, 19329, 14280, 16516, 1830...",510,"[8, 6, 9]"
200059,가스공-가스안전공-경동나비엔 등 취약계층 주거복지 통합 지원 협약,"가스공 올해 22억 투입, 복지시설 난방 개선 등한국가스공사는 10월 30일 경기도...",1009,10,"[18982, 9115, 244, 18982, 16239, 9115, 244, 90...","[[18982, 9115, 14516, 15271, 11764, 16490, 243...",405,"[8, 1, 5]"


In [18]:
df_aihub.to_csv("/<your_dir>/GSG_NAVER_200061.csv",index=False)